In [125]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [29]:
cwd = os.getcwd()
file_name = "MINDsmall_train/news.tsv"
file_path =  os.path.join(cwd, file_name)

In [30]:
news_df = pd.read_csv(file_path, sep='\t',header=None, names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' , 'URL', 'TitleEntities', 'AbstractEntities'])

In [31]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract,URL,TitleEntities,AbstractEntities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [66]:
news_df.drop(columns=['URL', 'TitleEntities' ,'AbstractEntities' ], inplace=True)

In [67]:
news_df.head()

,NewsID,Category,SubCategory,Title,Abstract
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."


In [59]:
import openai as OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

Python-dotenv could not parse statement starting at line 2


In [60]:
openai.api_key = OPENAI_API_KEY
openai_client = openai.OpenAI()

In [100]:
from pinecone import Pinecone, ServerlessSpec

INDEX_NAME = 'pc-index-news'
pinecone = Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=384, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [101]:
from sentence_transformers import SentenceTransformer


def get_embeddings(articles, model=SentenceTransformer('all-MiniLM-L6-v2')):
   return model.encode(articles)

In [122]:
news_df.describe()

,NewsID,Category,SubCategory,Title,Abstract
count,51282,51282,51282,51282,48616
unique,51282,17,264,50434,47309
top,N55528,news,newsus,Photos of the Day,What's the weather today? What's the weather f...
freq,1,15774,6564,15,124


In [133]:
from tqdm.auto import tqdm, trange

CHUNK_SIZE=100
TOTAL_ROWS=10000
progress_bar = tqdm(total=TOTAL_ROWS)
use_columns = [0,1,2,3,4]

chunks = pd.read_csv(file_path, sep='\t',header=None,usecols=use_columns , 
                     names=['NewsID', 'Category', 'SubCategory', 'Title' , 'Abstract' ]
                     , chunksize=CHUNK_SIZE, 
                     nrows=TOTAL_ROWS)



chunk_num = 0
for chunk in chunks:
    titles = chunk['Title'].tolist()
    print(f'{titles}/n')
    embeddings = get_embeddings(titles)
    prepped = [{'id':str(chunk_num*CHUNK_SIZE+i), 'values':embeddings[i],
                'metadata':{'title':titles[i]}} for i in range(0,len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        index.upsert(prepped)
        prepped = []
    progress_bar.update(len(chunk))

100%|██████████| 1000/1000 [08:56<00:00,  1.86it/s]


['The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By', '50 Worst Habits For Belly Fat', "The Cost of Trump's Aid Freeze in the Trenches of Ukraine's War", "I Was An NBA Wife. Here's How It Affected My Mental Health.", 'How to Get Rid of Skin Tags, According to a Dermatologist', 'Should NFL be able to fine players for criticizing officiating?', "It's been Orlando's hottest October ever so far, but cooler temperatures on the way", 'Chile: Three die in supermarket fire amid protests', 'Best PS5 games: top PlayStation 5 titles to look forward to', 'How to report weather-related closings, delays', '50 Foods You Should Never Eat, According to Health Experts', 'Trying to Make a Ram 3500 as Quick as a Viper Requires Some Disassembly', '25 Biggest Grocery Store Mistakes Making You Gain Weight', 'Instagram Filters with Plastic Surgery-Inspired Effects Could Soon Disappear', 'Michigan apple recall: Nearly 2,300 crates could be contaminated with listeria', "Kate Middleton's Bes

['14 Celebs Over 50 Who Are In The Best Shape Of Their Lives', 'The Oldsmobile Toronado - was it the ultimate personal car?', "With impeachment inquiry 'concerns' getting addressed, GOP have nowhere to go", '28 Surprisingly Good Costco, Walmart, and Target Products', "17 Low-Impact Workouts You Should Try If You're Not In Your 20s Anymore", 'When royals lose their tempers, from the Queen to Kate Middleton', '25 Hot Jobs That Pay More Than $100,000 a Year', "Queen Elizabeth's Cousin Says Royal Family 'Don't Communicate Very Well'", '12 ways your skin can react to stress, according to a dermatologist', 'Ikea US president backpedals on meeting parents of children killed by recalled dressers', 'Multi-vehicle accident on I-64 in James City County causes miles-long backup', 'Dissecting the softer, gentler Tiger Woods', '39 Appetizers for a Crowd That Are Easy and Unexpected', "How To Guess Someone's Zodiac Sign", 'What Men Find Attractive According To Their Zodiac Sign', 'Rams Week 8 snap co

['Martin Truex Jr. through the years', "5 Turkey Cooking Tips That'll Guarantee You Have the Perfect Bird", "Adrian Peterson had to 'hold back tears' in return to Minnesota", 'These Room Divider Ideas Will Transform Your Home in an Instant', "Buttigieg: 'Not going to get in the middle' of Hillary Clinton attacks on Tulsi Gabbard", 'I quit my job, moved to Alaska to bake bread   and rebuilt my identity', '10 Turkey Sandwich Recipes for All Your Thanksgiving Leftovers', 'Suez says lead levels meet federal standards, but only when you round down', '6 gorgeous royal family heirlooms that Kate Middleton wears regularly', 'Power Rankings: Kyle Busch putting pieces together before favorable Round of 8', 'What millennials get wrong about Social Security', 'Need the Perfect Last-Minute Gift? We Got You Covered', 'Celebrities at their Hollywood hand and footprint ceremonies', 'Wolverines in the NFL: Top performances from Week 8', 'Tanker truck bursts into flames following crash in Warren County'

["Here's what history says about Astros' chances of World Series comeback against Nationals", "President Trump's trillion-dollar hit to homeowners", 'This Might Be the 2020 Land Rover Defender', "Here's how to take over an auto lease", '10 Things Women Need To Stop Feeling So Guilty About', 'Doctors Want FDA To Warn Women About Cheese And Breast Cancer Link', 'The best fall day trip in every state', '50 Incredible Photos of the Swedish Royal Family Through the Years', "Exploring Chevy's First Big-Block in an Idaho Junkyard", 'Ultra-Luxe Bridal Shower Hostess Gift Ideas', "These are the best o' the best PC games available today", "Crockett Row adds 2 bar-restaurants, but also hooks a big one: Dallas' The Blue Fish", 'Ann Patchett spins a dark, compelling fairy tale in The Dutch House', '17 Easy Fall Slow-Cooker Recipes That Taste Like Autumn', '32 Best Good-Mood Foods', "'Souping': Should you try it?", 'The Best Dishes the Eater Boston Team Ate This Week', 'The Long, Hard Road: Find all

...

['Rapper Tekashi 6ix9ine offered massive record deal in prison', 'Report: Jets asked for Zack Martin, Tyron Smith in Jamal Adams trade, Cowboys offered Anthony Brown', "Last stop for Mt. Airy's Trolley Car Diner: 'It's been an amazing ride'", 'WikiLeaks founder Assange appears confused at extradition hearing', "Bebe Rexha inspired by Serena for 'Maleficent' song", "Man in good health gets 'DNR' tattoo in case of medical emergency", 'Why Are Democrats Jilting G.O.P. Voters Who Want to Like Them?', "Darnold trolled for 'ghost' comment with airplane banner in Jacksonville", "Early and pricey: Trump's World Series ad an expensive pitch", 'Officials hit Water Gremlin   again   with new round of pollution charges', "Pelosi's impeachment resolution turns into a massive fundraising day for Trump's 2020 campaign", "Trump's money machine triggers Democratic alarms", "Are Delta's New SkyMiles Membership Perks Worth the $59 Annual Fee?", 'License hearing for St. Louis Planned Parenthood set for Mo

['ABC Shuts Down Orange County Craft Distillery for 25 Days for Violating 1.5 Ounce Serving Limit', 'Bachelor in Paradise Stars Tayshia Adams and John Paul Jones Split', 'CTU Strike Continues, Fourth Day Of Canceled Classes For CPS Students', "Critics: $10-$28B Commuter Rail overhaul could be a 'New Big Dig'", 'Raid Gives Trump Well-Timed Win But Unlikely to Slow Impeachment', 'Jenna Dewan Gushes Over Boyfriend Steve Kazee in Touching Birthday Tribute', 'As Iraq and Lebanon protests flare, Iran clings to hard-earned sway', 'Family of man killed in Hard Rock collapse file wrongful death lawsuit', 'Diplomats describe all-time low in morale at State under Trump', "Microsoft's Secured-core PCs could be hacker-proof thanks to Xbox tech", 'Ecstatic ostrich spins, runs around outside Cincinnati Zoo', "Apple's iOS 13 is running on 50 percent of all iPhones after three weeks", 'Jason Reitman shares adorable cast photo as Ghostbusters 2020 wraps filming', 'These daily habits might keep you from 

['Seahawks place Tedric Thompson on injured reserve', 'FBI to investigate slaying of American prosecutor in Micronesia', 'What viral budgets reveal about the rich', "HP-per-liter: the world's most powerful car engines", 'RIDOT plans East Main Road safety audit', "Report: Arkansas has stopped paying Bielema's buyout", 'Tesla stock rallies 20% after surprise quarterly profit', "Report: Jets Open To Trading Le'Veon Bell If Price Is Right", 'Wire Taps: Nationals win pennant, sweep Cardinals; Howie Kendrick wins NLCS MVP; Mark Lerner says he wants to re-sign Anthony Rendon', "Five things we learned in the Chiefs' 31-24 loss to the Packers", "Transcripts show Republicans' scattershot strategy in early days of impeachment inquiry", 'A Complete Guide to Every Food You Can (and Cannot) Fly With', "Watch: Rams' Cooks out with concussion after scary hit to helmet", 'Turnovers propelled the Cowboys to victory against the Eagles last week', 'Palm Beach boy still missing a day later. Have you seen D

["Florida's attorney general launches investigation into 20 vaping companies", "Al-Baghdadi Is Dead. The Story Doesn't End Here.", 'Nationals Parade route for Saturday announced', '6 Surprising Sales to Shop on Black Friday', 'Local family donates $5 million to UD', 'These Celebrity Hair Transformations Make Us Want to Run to the Salon', 'Mike Tomlin open to considering massive coaching offer from Redskins?', 'Good Lord: As Eagles teeter on the brink of disaster a la T.O. and 2005, Wentz looks to the heavens | Marcus Hayes', "For Spurs' DeMar DeRozan, three's a conundrum", "Feds take down the world's 'largest dark web child porn marketplace'", '5 fun things to do this week Oct. 28-Nov. 1', 'Actually, President Trump, some Kurds did fight in World War II', 'Boxer From Long Island Dies After Suffering Traumatic Brain Injuries, Being Knocked Out In The Ring', "Black mold is growing in U.S. prisons, but federal officials won't test it", "Twin: She's my other half", "U.S. mulls extending Ch

['Pinellas deputy cleared in death of Guatemalan man he had stopped over a false report', 'Conway spars with Wallace on whether White House will cooperate with impeachment inquiry after formal vote', 'Happiness Is Key To Living A Longer Life', "Hundreds gather at dying Jacksonville veteran's bedside thanking him for service", 'Groups: Saving Mexican gray wolves requires new approach', 'Pope Francis accidentally blessed Saints, now Jaguars are doomed', "Go Inside Kris & Kylie Jenner's Opulent Homes", "Antoni Porowski's Journey to Self-Acceptance and Body Positivity Is Beautiful to Hear", 'How to Protect America After the Syria Withdrawal', 'World Series: Four things the Nationals must do to pull off one more October comeback', 'Woman charged with fatally shooting Kernersville man', "Mom gets to hear son's heartbeat again 5 years after his death", "NASA reveals 'galaxy of horrors' exoplanet posters in time for Halloween", 'City of Cleveland to commit $10 million for planting new trees', 

In [108]:
chunks

In [134]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [135]:
def get_recommendations(pinecone_index, search_term, top_k=10):
  embed = get_embeddings(search_term)
  res = pinecone_index.query(vector=embed.tolist(), top_k=top_k, include_metadata=True)
  return res

In [138]:
reco = get_recommendations(index, 'weather')
for r in reco.matches:
    print(f'{r.score} : {r.metadata["title"]}')

0.521648228 : 4Warn Forecast: Showers moving out, chilly for the Cardinals game tonight!
0.519576609 : New York Weather: Winter Expected To Be Warmer & Wetter Than Normal, NOAA Says
0.425071329 : Denver Weather: Second Storm Hits Sooner Than Expected With More Snow, Record Cold
0.419349939 : Nestor bringing heavy rain to Southeast, mid-Atlantic after spawning tornadoes in Florida
0.409070522 : It's been Orlando's hottest October ever so far, but cooler temperatures on the way
0.379096627 : South-central US to face nighttime severe storms, isolated tornadoes at end of weekend
0.375454485 : How to report weather-related closings, delays
0.369575381 : Winter Storm Warning For Metro Denver, Boulder: Update
0.367477238 : National Weather Service To Visit Madison, NJ
0.350768 : Photos of the Day
